In [2]:
#!conda install -c conda-forge geopy --yes
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup as bs
from geopy.geocoders import Nominatim

# we try to get Chinatown location first

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
neighborhoods_data = newyork_data['features']
#take a look at structure in feature
newyork_data['features'][1]

{'type': 'Feature',
 'id': 'nyu_2451_34572.2',
 'geometry': {'type': 'Point',
  'coordinates': [-73.82993910812398, 40.87429419303012]},
 'geometry_name': 'geom',
 'properties': {'name': 'Co-op City',
  'stacked': 2,
  'annoline1': 'Co-op',
  'annoline2': 'City',
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.82993910812398,
   40.87429419303012,
   -73.82993910812398,
   40.87429419303012]}}

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
#get all info into dataframe
for x in neighborhoods_data:
    borough = x['properties']['borough'] 
    neighborhood_name = x['properties']['name']
        
    neighborhood_latlon = x['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head(20)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [9]:
neighborhoods.loc[neighborhoods['Neighborhood']=='Chinatown']

,Borough,Neighborhood,Latitude,Longitude
100,Manhattan,Chinatown,40.715618,-73.994279


In [10]:
#get chinatown latitude and longtitude
latitude=neighborhoods.iloc[100,-2]
longitude=neighborhoods.iloc[100,-1]
print('Chinatown in New York is at ({},{})'.format(latitude,longitude))

Chinatown in New York is at (40.71561842231432,-73.99427936255978)


In [11]:
#next, use explore function in Foursquare API to find 100 Chinese Restaurant

In [12]:
#Although we can use parameter 'categoryId' to filter all Chinese Restaurant, but we increase difficulty by manually filtering in another endpoint 'VenueDetails'
CLIENT_ID='0BPBI1NPMF11CVC2IP2AOOGYLSCFZIZCHTN4UKHQJCCFVHFN'
CLIENT_SECRET='5S1BMTKQ50OSBHWTO1EAHRBTOYBXPGZLIR1G1SZUFIBU103F'
VERSION='20200701'
radius=5000
LIMIT=200
categoryId='4d4b7105d754a06374d81259'
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT,categoryId)
url

'https://api.foursquare.com/v2/venues/explore?client_id=0BPBI1NPMF11CVC2IP2AOOGYLSCFZIZCHTN4UKHQJCCFVHFN&client_secret=5S1BMTKQ50OSBHWTO1EAHRBTOYBXPGZLIR1G1SZUFIBU103F&ll=40.71561842231432,-73.99427936255978&v=20200701&radius=5000&limit=200&categoryId=4d4b7105d754a06374d81259'

In [13]:
results = requests.get(url).json()

In [14]:
df_init=[]

In [15]:
columnname=['NAME','ID','ADDRESS']

In [16]:
df_init=pd.DataFrame(columns=columnname)
df_init

,NAME,ID,ADDRESS


In [17]:
#use for loop to add all results into dataframe, note that due to limit by API, we can only retrive 100 spots at one time
for item in results['response']['groups'][0]['items']:
    NAME=item['venue']['name']
    ID=item['venue']['id']
    try:
        ADDRESS=item['venue']['location']['address']
    except:
        ADDRESS='N/A'
    df_init=df_init.append({'NAME':NAME,'ID':ID,'ADDRESS':ADDRESS},ignore_index=True)
df_init

,NAME,ID,ADDRESS
0,Cheeky Sandwiches,4b1896caf964a52069d423e3,35 Orchard St
1,Kiki's,5521c2ff498ebe2368634187,130 Division St
2,Scarr's Pizza,56c3c626cd106998d2b196d0,22 Orchard St
3,Wayla,5cc4e9d0c876c8002c3010cb,100 Forsyth St
4,Forgtmenot,4fd38a04e4b065401a9aaf88,138 Division St
...,...,...,...
95,Jeffrey's Grocery,4ca21faa8afca09388192116,172 Waverly Pl
96,Buvette,4d0bf7e3f29c236ac675bfe7,42 Grove St
97,L’Accolade,5d181bac17d3b9002329f126,302 Bleecker St
98,CAVA,57bcd600498e81d153625a1a,143 4th Ave


In [17]:
#now we use id in each row and Premium endpoint of Foursquare API ' https://api.foursquare.com/v2/venues/VENUE_ID' to search for categories,price tier,likes count,rating,
#ratingSignals,likes count,popular timeframe,latitude,longitude
#1. we use a function to find out Chinese Restaurant and Non-Chinese Restaurant 
#2. we use first for loop to filter the most liked tip
#3. we use second for loop to combine all popular timeframe into one,cos timeframes can be more than one

In [18]:
#1. we use a function to find out Chinese Restaurant and Non-Chinese Restaurant,cos one venue can have more than one categories, so we use for loop to filter all categories
def if_cnr(venue_input):
    
        list1=[]
        try:
            for item in venue_input['response']['venue']['categories']:
                x=('4bf58dd8d48988d145941735' in item['id'])
                list1.append(x)
            if True in list1:
                return 'Chinese Restaurant'
            else:
                return 'Non-Chinese Restaurant'
        except:
            return 'N/A'

In [20]:
#Use one venue id to take a look at its structure
url_venue='https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format('585164b77220e62219c9aeb6',CLIENT_ID, CLIENT_SECRET,VERSION)
print(url_venue)
venue_detail=requests.get(url_venue).json()
venue_detail['response']['venue']

https://api.foursquare.com/v2/venues/585164b77220e62219c9aeb6?&client_id=0BPBI1NPMF11CVC2IP2AOOGYLSCFZIZCHTN4UKHQJCCFVHFN&client_secret=5S1BMTKQ50OSBHWTO1EAHRBTOYBXPGZLIR1G1SZUFIBU103F&v=20200701


{'id': '585164b77220e62219c9aeb6',
 'name': '4 Charles Prime Rib',
 'contact': {'phone': '2125615992',
  'formattedPhone': '(212) 561-5992',
  'facebook': '885277744922395',
  'facebookUsername': '4charlesprimerib',
  'facebookName': '4 Charles Prime Rib'},
 'location': {'address': '4 Charles St',
  'crossStreet': 'Greenwich Ave',
  'lat': 40.735219495923204,
  'lng': -74.0006488188871,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.735219495923204,
    'lng': -74.0006488188871},
   {'label': 'entrance', 'lat': 40.735185, 'lng': -74.000661}],
  'postalCode': '10014',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['4 Charles St (Greenwich Ave)',
   'New York, NY 10014',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/4-charles-prime-rib/585164b77220e62219c9aeb6',
 'categories': [{'id': '4bf58dd8d48988d1cc941735',
   'name': 'Steakhouse',
   'pluralName': 'Steakhouses',
   'shortName': 'Steakhouse',
   '

In [ ]:
#testing tips API
url_tips='https://api.foursquare.com/v2/venues/{}/tips?&sort=popular&client_id={}&client_secret={}&v={}'.format('4fd38a04e4b065401a9aaf88',CLIENT_ID, CLIENT_SECRET,VERSION)
print(url_tips)
tips_detail=requests.get(url_tips).json()
tips_detail['response']['tips']['items'][0]['agreeCount']

#2. we use first for loop to filter the most liked tip
#3. we use second for loop to combine all popular timeframe into one,cos timeframes can be more than one
#4. Due to venues details are not the same, for example, some have tips some don't, so we use try/except to catach some details in case errors occur

df=df_init
df['Category']=''
df['Price_Tier']=''
df['Likes_Count']=''
df['Rating']=''
df['Rating_Signals']=''
df['Tips']=''
df['Agree_Count']=''
df['Polular_Timeframe_Today']=''
df['Latitude']=''
df['Longitude']=''

for index,row in df_init.iterrows():
    id_venue=row['ID']
    url_venue='https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(id_venue,CLIENT_ID, CLIENT_SECRET,VERSION)
    url_tips='https://api.foursquare.com/v2/venues/{}/tips?&sort=popular&client_id={}&client_secret={}&v={}'.format(id_venue,CLIENT_ID, CLIENT_SECRET,VERSION)
    venue_detail=requests.get(url_venue).json()
    tips_detail=requests.get(url_tips).json()
    #deal with categories function
    Category=if_cnr(venue_detail)
    #deal with other parameters
    try:
        Price_Tier=venue_detail['response']['venue']['price']['tier']
    except:
        Price_Tier='N/A'
    try:
        Likes_Count=venue_detail['response']['venue']['likes']['count']
    except:
        Likes_Count='N/A'
    try:
        Rating=venue_detail['response']['venue']['rating']
    except:
        Rating='N/A'
    try:
        Rating_Signals=venue_detail['response']['venue']['ratingSignals']
    except:
        Rating_Signals='N/A'
    #use API parameter 'sort=popular' to retrive most liked tips
    Tips=''
    Agree_Count=0
    try:
        Tips=tips_detail['response']['tips']['items'][0]['text']
        Agree_Count=tips_detail['response']['tips']['items'][0]['agreeCount']
    except:
        Agree_Count='N/A'
        Tips=='N/A'

    #deal with today's popular timeframes,combine all timeframe into one
    Polular_Timeframe_Today=''
    try:
        for y in venue_detail['response']['venue']['popular']['timeframes'][0]['open']:
            Polular_Timeframe_Today+=y['renderedTime']
    except:
        Polular_Timeframe_Today='N/A'
    
    try:
        Latitude=venue_detail['response']['venue']['location']['lat']
        Longitude=venue_detail['response']['venue']['location']['lng']
    except:
        Latitude='N/A'
        Longitude='N/A'
    #all retrived data into columns
    row['Category']=Category
    row['Price_Tier']=Price_Tier
    row['Likes_Count']=Likes_Count
    row['Rating']=Rating
    row['Rating_Signals']=Rating_Signals
    row['Tips']=Tips
    row['Agree_Count']=Agree_Count
    row['Polular_Timeframe_Today']=Polular_Timeframe_Today
    row['Latitude']=Latitude
    row['Longitude']=Longitude


In [74]:
# show dataframe
df.head()

,NAME,ID,ADDRESS,Category,Price_Tier,Likes_Count,Rating,Rating_Signals,Tips,Agree_Count,Polular_Timeframe_Today,Latitude,Longitude
0,Cheeky Sandwiches,4b1896caf964a52069d423e3,35 Orchard St,Non-Chinese Restaurant,2,545,9.2,739,The chicken sandwich is a must try. The beigne...,5,10:00 AM–10:00 PM,40.7158,-73.9918
1,Kiki's,5521c2ff498ebe2368634187,130 Division St,Non-Chinese Restaurant,2,830,9.1,1066,Unbelievable. Hands down the best Greek food o...,18,Noon–Midnight,40.7145,-73.992
2,Scarr's Pizza,56c3c626cd106998d2b196d0,22 Orchard St,Non-Chinese Restaurant,N/A,229,9.1,294,This place has an old school diner vibe with d...,12,Noon–Midnight,40.7153,-73.9916
3,Wayla,5cc4e9d0c876c8002c3010cb,100 Forsyth St,Non-Chinese Restaurant,N/A,127,9.3,147,Get the branzino if you like flavors like tama...,8,11:00 AM–3:00 PM5:00 PM–11:00 PM,40.7183,-73.9926
4,Forgtmenot,4fd38a04e4b065401a9aaf88,138 Division St,Non-Chinese Restaurant,2,517,9,646,"After careful review of the LES, this might be...",6,Noon–1:00 AM,40.7145,-73.9915


In [35]:
# take out rating=N/A items
df1=df[df['Rating']!='N/A']
df1

,NAME,ID,ADDRESS,Category,Price_Tier,Likes_Count,Rating,Rating_Signals,Tips,Agree_Count,Polular_Timeframe_Today,Latitude,Longitude
0,Cheeky Sandwiches,4b1896caf964a52069d423e3,35 Orchard St,Non-Chinese Restaurant,2,545,9.2,739,The chicken sandwich is a must try. The beigne...,5,10:00 AM–10:00 PM,40.7158,-73.9918
1,Kiki's,5521c2ff498ebe2368634187,130 Division St,Non-Chinese Restaurant,2,830,9.1,1066,Unbelievable. Hands down the best Greek food o...,18,Noon–Midnight,40.7145,-73.992
2,Scarr's Pizza,56c3c626cd106998d2b196d0,22 Orchard St,Non-Chinese Restaurant,N/A,229,9.1,294,This place has an old school diner vibe with d...,12,Noon–Midnight,40.7153,-73.9916
3,Wayla,5cc4e9d0c876c8002c3010cb,100 Forsyth St,Non-Chinese Restaurant,N/A,127,9.3,147,Get the branzino if you like flavors like tama...,8,11:00 AM–3:00 PM5:00 PM–11:00 PM,40.7183,-73.9926
4,Forgtmenot,4fd38a04e4b065401a9aaf88,138 Division St,Non-Chinese Restaurant,2,517,9,646,"After careful review of the LES, this might be...",6,Noon–1:00 AM,40.7145,-73.9915
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Jeffrey's Grocery,4ca21faa8afca09388192116,172 Waverly Pl,Non-Chinese Restaurant,3,685,9.2,907,I love here!!! Their Bloody May is the best in...,5,10:00 AM–1:00 AM,40.7339,-74.0013
96,Buvette,4d0bf7e3f29c236ac675bfe7,42 Grove St,Non-Chinese Restaurant,3,1849,9.1,2373,"Amazing place and delicious food, just a bit p...",16,9:00 AM–11:00 PM,40.7328,-74.0043
97,L’Accolade,5d181bac17d3b9002329f126,302 Bleecker St,Non-Chinese Restaurant,N/A,23,9,45,Potatoes are sooo good. Plus $10 rotating wine...,1,4:00 PM–Midnight,40.7325,-74.0038
98,CAVA,57bcd600498e81d153625a1a,143 4th Ave,Non-Chinese Restaurant,1,330,9,416,"The salad bowls are a superior tasting, health...",13,Noon–4:00 PM7:00 PM–8:00 PM,40.7338,-73.9896


In [36]:
#sort by rating for next analysis
df2=df1.sort_values(by='Rating', ascending=False)
df2

,NAME,ID,ADDRESS,Category,Price_Tier,Likes_Count,Rating,Rating_Signals,Tips,Agree_Count,Polular_Timeframe_Today,Latitude,Longitude
70,Brooklyn Bagel & Coffee Company,5b8e66ac234724002c927c4b,63 E 8th St,Non-Chinese Restaurant,N/A,78,9.4,91,"Freshly made, great choice and delicious! So g...",4,7:00 AM–3:00 PM,40.7309,-73.9933
76,Faicco's Italian Specialties,4a74a36af964a520fede1fe3,260 Bleecker St,Non-Chinese Restaurant,2,332,9.4,445,Order the Italian special ($12) features: pros...,8,10:00 AM–6:00 PM,40.7311,-74.003
99,4 Charles Prime Rib,585164b77220e62219c9aeb6,4 Charles St,Non-Chinese Restaurant,N/A,217,9.3,270,Get the ribeye and Tempranillo. Split the burg...,8,5:00 PM–Midnight,40.7352,-74.0006
38,SUGARFISH by sushi nozawa,5c6c6715f709c1002cd49290,202 Spring St,Non-Chinese Restaurant,N/A,83,9.3,94,Opens 2/21!,3,Noon–11:00 PM,40.7253,-74.0035
46,Sunny & Annie Gourmet Deli,4b41102bf964a52055c025e3,94 Avenue B,Non-Chinese Restaurant,1,429,9.3,571,Life altering sandwiches. I had the 2014 with ...,4,N/A,40.7246,-73.9816
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,Saigon Vietnamese Sandwich Deli,49e9549df964a520f2651fe3,369 Broome St,Non-Chinese Restaurant,1,440,8.9,631,The best banh mi sandwich in NYC! If you are a...,3,11:00 AM–6:00 PM,40.7201,-73.9957
67,Banter,589c7ca37b43b441e3ac3ef8,169 Sullivan St,Non-Chinese Restaurant,N/A,274,8.9,351,Adorable new Australian café on Sullivan St! P...,10,9:00 AM–5:00 PM7:00 PM–9:00 PM,40.728,-74.0012
94,Loring Place,58324f3f9dc8d00516f64b6d,21 W 8th St,Non-Chinese Restaurant,N/A,404,8.9,503,Everything is fantastic! Exactly what you'd ex...,14,11:00 AM–3:00 PM5:00 PM–11:00 PM,40.7329,-73.9976
26,Sweet Chick,537b5a29498ec121cf9fa1f4,178 Ludlow St,Non-Chinese Restaurant,2,678,8.9,893,Want something sweet for brunch? Get the Chick...,12,10:00 AM–11:00 PM,40.7218,-73.9875


In [37]:
#we only trace top 20 rating places
df3=df2[0:21]
df3

,NAME,ID,ADDRESS,Category,Price_Tier,Likes_Count,Rating,Rating_Signals,Tips,Agree_Count,Polular_Timeframe_Today,Latitude,Longitude
70,Brooklyn Bagel & Coffee Company,5b8e66ac234724002c927c4b,63 E 8th St,Non-Chinese Restaurant,N/A,78,9.4,91,"Freshly made, great choice and delicious! So g...",4,7:00 AM–3:00 PM,40.7309,-73.9933
76,Faicco's Italian Specialties,4a74a36af964a520fede1fe3,260 Bleecker St,Non-Chinese Restaurant,2,332,9.4,445,Order the Italian special ($12) features: pros...,8,10:00 AM–6:00 PM,40.7311,-74.003
99,4 Charles Prime Rib,585164b77220e62219c9aeb6,4 Charles St,Non-Chinese Restaurant,N/A,217,9.3,270,Get the ribeye and Tempranillo. Split the burg...,8,5:00 PM–Midnight,40.7352,-74.0006
38,SUGARFISH by sushi nozawa,5c6c6715f709c1002cd49290,202 Spring St,Non-Chinese Restaurant,N/A,83,9.3,94,Opens 2/21!,3,Noon–11:00 PM,40.7253,-74.0035
46,Sunny & Annie Gourmet Deli,4b41102bf964a52055c025e3,94 Avenue B,Non-Chinese Restaurant,1,429,9.3,571,Life altering sandwiches. I had the 2014 with ...,4,N/A,40.7246,-73.9816
52,Crown Shy,5c883f65f4b525002c0bf2ca,70 Pine St,Non-Chinese Restaurant,N/A,154,9.3,187,You have to drink the Sinclaire and my favorit...,10,Noon–1:00 PM5:00 PM–11:00 PM,40.7062,-74.0075
9,Le Coucou,57583641498e90001a32e13e,138 Lafayette St,Non-Chinese Restaurant,3,394,9.3,498,"At Le Coucou, the menu includes seaweed butter...",4,10:00 AM–3:00 PM5:00 PM–11:00 PM,40.7191,-74.0002
32,Los Tacos No. 1,5d5f24ec09484500079aee00,136 Church St,Non-Chinese Restaurant,N/A,78,9.3,95,The new taco joint in town (or in TriBeCa area...,9,11:00 AM–9:00 PM,40.7143,-74.0088
3,Wayla,5cc4e9d0c876c8002c3010cb,100 Forsyth St,Non-Chinese Restaurant,N/A,127,9.3,147,Get the branzino if you like flavors like tama...,8,11:00 AM–3:00 PM5:00 PM–11:00 PM,40.7183,-73.9926
71,Shake Shack,59d36de20fe7a024363de0b8,20 3rd Ave,Non-Chinese Restaurant,N/A,163,9.2,191,The best Shake Shack in the city. All orders a...,21,Noon–Midnight,40.73,-73.9897


In [38]:
#install and import folium function
#! conda install -c conda-forge folium
import folium 
import webbrowser
from folium.plugins import MarkerCluster

In [66]:
# create map of New York using latitude and longitude values
# https://python-visualization.github.io/folium/quickstart.html
map_chinatown = folium.Map(location=[latitude, longitude], zoom_start=14)
map_chinatown

In [67]:
# add markers to map
for name, lat, lng, rating, address,price_tier, Polular_Timeframe_Today in zip(df3['NAME'],df3['Latitude'], df3['Longitude'], df3['Rating'], df3['ADDRESS'],df3['Price_Tier'],df3['Polular_Timeframe_Today']):
    label = 'Rating:{}, Address:{}, Price_Tier:{},\n Avoid Time:{}'.format(rating,address,price_tier,Polular_Timeframe_Today )
    tooltip= '{}'.format(name)
    if price_tier==1 or price_tier==2:
        folium.Marker([lat, lng],popup=label,icon=folium.Icon(color='blue',icon='info-sign'),tooltip=tooltip).add_to(map_chinatown)
    elif price_tier==3 or price_tier==4:
        folium.Marker([lat, lng],popup=label,icon=folium.Icon(color='red',icon='info-sign'),tooltip=tooltip).add_to(map_chinatown)
    else:
        folium.Marker([lat, lng],popup=label,icon=folium.Icon(color='black',icon='info-sign'),tooltip=tooltip).add_to(map_chinatown)
        
map_chinatown


In [72]:
# add clusters into map
map_chinatown = folium.Map(location=[latitude, longitude], zoom_start=14)

In [73]:
marker_cluster = folium.plugins.MarkerCluster().add_to(map_chinatown) 
for index, row in df3.iterrows():
    locationlist = row[['Latitude','Longitude']].values.tolist()
    tooltip= '{}'.format(name)
    if row['Price_Tier']==1 or row['Price_Tier']==2:
        folium.Marker(location=locationlist, popup='Address:{}, Rating:{}, Price Tier:{},/n Avoid Time:{}'.format(row['ADDRESS'],row['Rating'],row['Price_Tier'],row['Polular_Timeframe_Today']), icon=folium.Icon(color='green', icon='ok-sign'),tooltip=tooltip).add_to(marker_cluster)
    elif row['Price_Tier']==3 or row['Price_Tier']==4:
        folium.Marker(location=locationlist, popup='Address:{}, Rating:{}, Price Tier:{},/n Avoid Time:{}'.format(row['ADDRESS'],row['Rating'],row['Price_Tier'],row['Polular_Timeframe_Today']), icon=folium.Icon(color='red', icon='ok-sign'),tooltip=tooltip).add_to(marker_cluster)
    else:
        folium.Marker(location=locationlist, popup='Address:{}, Rating:{}, Price Tier:{},/n Avoid Time:{}'.format(row['ADDRESS'],row['Rating'],row['Price_Tier'],row['Polular_Timeframe_Today']), icon=folium.Icon(color='black', icon='ok-sign'),tooltip=tooltip).add_to(marker_cluster)
map_chinatown
    

In [ ]:
# Finally,we have:
# 1. A intuitive map with top 20 rating restaurants around Chinatown in NY
# 2. If there are more than two restaurants nearby one location, there will be clucters button to click
# 3. We can mouseover to those locations to see their names
# 4. We can choose different price tiers by looking at their colors, green means cheap, red means a bit expensive, black means we dont have price info of these locations
# 5. By clicking those locations we can see details of address,rating,price tier and popular time frame to avoid queuing